In [1]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
RAND_STATE = 42

In [3]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values

# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values
    
class PolicyNetwork(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=0.5, pre_trained_embeddings=None):
        super(PolicyNetwork, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        logits = self.fc(out)
        probabilities = self.softmax(logits)
        return probabilities

def policy_gradient_loss(probabilities, actions, rewards):
    # Negative log likelihood loss
    log_prob = torch.log(probabilities[range(len(actions)), actions])
    loss = -torch.mean(log_prob * rewards)
    return loss

In [4]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, actions) in enumerate(train_loader):
        data, actions = data.to(device), actions.to(device)
        optimizer.zero_grad()
        probabilities = model(data)
        
        # Simulate rewards; in a real RL scenario, this would come from the environment
        rewards = torch.rand(len(actions))  # Placeholder for actual reward calculation logic
        
        loss = policy_gradient_loss(probabilities, actions, rewards)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [5]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[ 3,  2,  1, 18,  0,  0,  0,  0,  0,  0],
        [50, 47, 60, 27,  0,  0,  0,  0,  0,  0],
        [48,  4, 68, 63,  0,  0,  0,  0,  0,  0],
        [13, 51, 38, 48, 49,  0,  0,  0,  0,  0]])
Train Label: tensor([0, 1, 0, 0])
Validation Encoded text: tensor([[23, 20,  1,  0,  0,  0,  0,  0,  0,  0],
        [14, 13, 12, 18,  0,  0,  0,  0,  0,  0],
        [14, 30,  7,  2, 26, 29,  0,  0,  0,  0],
        [ 3, 15,  5,  4,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[ 7, 26,  2, 12, 11,  0,  0,  0,  0,  0],
        [ 1, 19,  9, 29,  1, 13, 31,  0,  0,  0],
        [ 4, 18, 27,  0,  0,  0,  0,  0,  0,  0],
        [23,  8, 30,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [6]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PolicyNetwork(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.535845

Validation set: Average loss: 0.1982, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.223305

Validation set: Average loss: 0.2145, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.256315

Validation set: Average loss: 0.2061, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.423716

Validation set: Average loss: 0.1977, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.329831

Validation set: Average loss: 0.2083, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.215342

Validation set: Average loss: 0.2201, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.355237

Validation set: Average loss: 0.2235, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.324944

Validation set: Average loss: 0.2172, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.135968

Validation set: Average loss: 0.2114, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.152913

Validation set: Average loss: 0.1972, 

In [8]:
# Parameters and Hyperparameters
n_trials=50
num_classes = 2

def objective(trial):
    # Define the search space
    vocabulary_size = trial.suggest_categorical('vocabulary_size', [5000, 10000, 20000, 40000])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    step_size = trial.suggest_int('step_size', 1, 100)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, log=True)
    sequence_length = trial.suggest_categorical('sequence_length', [50, 100, 200, 400])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = PolicyNetwork(vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=dropout_rate).to(device)

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
        scheduler.step()

    # Set custom attributes for the trial
    trial.set_user_attr("val_loss", val_loss)
    trial.set_user_attr("val_accuracy", val_accuracy)
    
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    # return val_loss

    # Objective: maximize validation accuracy by minimizing its negative value
    return -val_accuracy  # Return the negative accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

# Retrieve the validation loss and accuracy from the best trial
best_val_loss = trial.user_attrs["val_loss"]
best_val_accuracy = trial.user_attrs["val_accuracy"]

print(f'Best Validation Loss: {best_val_loss}')
print(f'Best Validation Accuracy: {best_val_accuracy}')
print('Best Trial Parameters:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-04-02 19:39:31,518] A new study created in memory with name: no-name-a894a04f-7a47-425a-b198-7357074a571e


Train Epoch: 1 [0/24 (0%)]	Loss: 0.307140

Validation set: Average loss: 0.1447, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.408744

Validation set: Average loss: 0.0609, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.547665

Validation set: Average loss: 0.1117, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.321245

Validation set: Average loss: 0.0951, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.357429

Validation set: Average loss: 0.0985, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.269746

Validation set: Average loss: 0.1125, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.305538

Validation set: Average loss: 0.0784, Accuracy: 7/8 (88%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.307839


[I 2024-04-02 19:39:31,856] Trial 0 finished with value: -0.125 and parameters: {'vocabulary_size': 20000, 'batch_size': 128, 'learning_rate': 0.005929030467010251, 'embedding_dim': 256, 'hidden_dim': 32, 'optimizer': 'RMSprop', 'dropout_rate': 0.15879446312530554, 'step_size': 59, 'gamma': 0.6978961839771177, 'sequence_length': 50}. Best is trial 0 with value: -0.125.



Validation set: Average loss: 0.1173, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.279405

Validation set: Average loss: 0.0863, Accuracy: 4/8 (50%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.200809

Validation set: Average loss: 0.1161, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.356076

Validation set: Average loss: 0.0905, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.284174

Validation set: Average loss: 0.0926, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.352302

Validation set: Average loss: 0.0931, Accuracy: 1/8 (12%)



[I 2024-04-02 19:39:32,205] Trial 1 finished with value: -0.125 and parameters: {'vocabulary_size': 10000, 'batch_size': 256, 'learning_rate': 2.5713472683837465e-05, 'embedding_dim': 256, 'hidden_dim': 128, 'optimizer': 'RMSprop', 'dropout_rate': 0.3157776415500287, 'step_size': 32, 'gamma': 0.9606166782064912, 'sequence_length': 50}. Best is trial 0 with value: -0.125.


...

Train Epoch: 6 [0/24 (0%)]	Loss: 0.366993

Validation set: Average loss: 0.0895, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.337782

Validation set: Average loss: 0.0898, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.368275

Validation set: Average loss: 0.0900, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.282576

Validation set: Average loss: 0.0903, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.267448

Validation set: Average loss: 0.0906, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.373638

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.286169

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.324015

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.292202

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.320394

Validation set: Average loss: 0.0879, 

[I 2024-04-02 19:39:42,766] Trial 47 finished with value: -0.125 and parameters: {'vocabulary_size': 40000, 'batch_size': 128, 'learning_rate': 0.0007160896438410186, 'embedding_dim': 64, 'hidden_dim': 64, 'optimizer': 'SGD', 'dropout_rate': 0.23276946935703152, 'step_size': 58, 'gamma': 0.15621045035193684, 'sequence_length': 200}. Best is trial 17 with value: -0.875.
[I 2024-04-02 19:39:42,930] Trial 48 finished with value: -0.125 and parameters: {'vocabulary_size': 10000, 'batch_size': 32, 'learning_rate': 0.002474629120096241, 'embedding_dim': 64, 'hidden_dim': 32, 'optimizer': 'Adam', 'dropout_rate': 0.12827147751411205, 'step_size': 50, 'gamma': 0.9777487234473025, 'sequence_length': 50}. Best is trial 17 with value: -0.875.


Train Epoch: 7 [0/24 (0%)]	Loss: 0.348798

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.313800

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.276350

Validation set: Average loss: 0.0879, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.428737

Validation set: Average loss: 0.0880, Accuracy: 1/8 (12%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.368984

Validation set: Average loss: 0.0921, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.326595

Validation set: Average loss: 0.0980, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.323809

Validation set: Average loss: 0.1036, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.323870

Validation set: Average loss: 0.1080, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.273149

Validation set: Average loss: 0.1111, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.249296

Validation set: Average loss: 0.1122, 

[I 2024-04-02 19:39:43,059] Trial 49 finished with value: -0.875 and parameters: {'vocabulary_size': 20000, 'batch_size': 256, 'learning_rate': 0.000276474295129079, 'embedding_dim': 64, 'hidden_dim': 32, 'optimizer': 'SGD', 'dropout_rate': 0.42437324790018155, 'step_size': 26, 'gamma': 0.32059089187416634, 'sequence_length': 100}. Best is trial 17 with value: -0.875.


Train Epoch: 1 [0/24 (0%)]	Loss: 0.364339

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.363235

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.343006

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.342138

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.363064

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.378890

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.383858

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.367974

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.358391

Validation set: Average loss: 0.0846, Accuracy: 7/8 (88%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.366093

Validation set: Average loss: 0.0846, 